In [1]:
import pandas as pd
import yaml
import numpy as np
import ast
from plbenchmark import edges, ligands, targets


# Define the custom orders
SYSTEM_ORDER = ["p38", "A2A", "ptp1b", "tyk2", "thrombin", "mcl1", "CyclophilinD", "SAMPL6-OA"]


def get_magnitude(Quantity):
    # The data is in kcal/mol
    return Quantity.magnitude

def get_plb_data(data_set, get_edges: bool = False):
    targets.set_data_dir('../../../experimental')
    ligands_set = ligands.LigandSet(data_set)
    df = ligands_set.get_dataframe()
    ref_data = pd.DataFrame()
    ref_data['ligand'] = df.name
    ref_data['exp_dG'] = df.DerivedMeasurement.value
    ref_data['exp_dG_error'] = df.DerivedMeasurement.error
    ref_data['exp_dG'] = ref_data['exp_dG'].map(get_magnitude)
    ref_data['exp_dG_error'] = ref_data['exp_dG_error'].map(get_magnitude).apply(lambda x: max(0, x))
    if get_edges:
        edges_df = edges.EdgeSet(target=data_set).get_dataframe()[['ligand_a', 'ligand_b', 'exp. DeltaG [kcal/mol]', 'exp. Error [kcal/mol]']]
        edges_df.rename(columns={'exp. DeltaG [kcal/mol]': 'exp_dG', 'exp. Error [kcal/mol]': 'exp_dG_error'}, inplace=True)
        edges_df.exp_dG = edges_df.exp_dG.map(get_magnitude)
        edges_df.exp_dG_error = edges_df.exp_dG_error.map(get_magnitude)
        return ref_data, edges_df
    else:
        return ref_data


def get_mbar_data():
    with open('../individual/systems_info_fep.yml', 'r') as f:    
        systems = yaml.safe_load(f)
        for system in SYSTEM_ORDER:
            for ff in systems[system]:
                if isinstance(systems[system][ff], list):
                    systems[system][ff] = [simu.replace('.csv', '_raw.csv') for simu in systems[system][ff]]
                else:
                    systems[system][ff] = systems[system][ff].replace('.csv', '_raw.csv')

    BindFlowData = pd.DataFrame()
    for system in systems:
        for ff in systems[system]:
            if isinstance(systems[system][ff], str):
                tmp_df = pd.read_csv(systems[system][ff], index_col=0)
                tmp_df['system'] = system
                tmp_df['simulation'] = f"simulation_mbar_{ff}"
            else:
                raise RuntimeError

            BindFlowData = pd.concat([BindFlowData, tmp_df], ignore_index=True)
    
    BindFlowData["value"] = BindFlowData["mbar_ligand_vdw_value"] + BindFlowData["mbar_ligand_coul_value"] - BindFlowData["boresch"] + \
        BindFlowData["mbar_complex_vdw_value"] + BindFlowData["mbar_complex_coul_value"] - BindFlowData["mbar_complex_bonded_value"]
    columns_to_kept = ['system', 'simulation','ligand', 'replica', 'value']
    BindFlowData = BindFlowData[columns_to_kept]

    BindFlowData = BindFlowData.pivot(index=["system", "ligand", "replica"],columns="simulation", values="value").reset_index()
    # Remove the columns' name
    BindFlowData.columns.name = None
    BindFlowData["sample"] = 1
    BindFlowData.sort_values(by=['system', 'ligand', "replica"], inplace=True)
    return BindFlowData


def get_mmxbsa_data():
    columns = {
        'name': 'ligand',
        'replica': 'replica',
        'sample': 'sample',
        'dg_c2_pb': 'simulation_dg-c2-pb',
        'dg_c2_gb': 'simulation_dg-c2-gb',
        'dg_ie_pb': 'simulation_dg-ie-pb',
        'dg_ie_gb': 'simulation_dg-ie-gb',
        'dg_en_pb': 'simulation_dh-pb',
        'dg_en_gb': 'simulation_dh-gb',
    }

    with open('../individual/systems_info_mmpbsa.yml', 'r') as f:    
        systems = yaml.safe_load(f)
        for system in SYSTEM_ORDER:
            for ff in systems[system]:
                if isinstance(systems[system][ff], list):
                    systems[system][ff] = [simu.replace('.csv', '_raw.csv') for simu in systems[system][ff]]
                else:
                    systems[system][ff] = systems[system][ff].replace('.csv', '_raw.csv')


    BindFlowData = pd.DataFrame()
    for system in systems:
        for ff in systems[system]:
            tmp_columns = columns.copy()
            for column in tmp_columns:
                if column not in ["name", "replica", "sample"]:
                    tmp_columns[column] = f"{tmp_columns[column]}_{ff}"
            if isinstance(systems[system][ff], str):
                tmp_df = pd.read_csv(systems[system][ff], usecols=tmp_columns.keys()).rename(columns=tmp_columns)
                
                tmp_df['system'] = system
                
                # Transform the wide-format DataFrame back to long format
                tmp_df = pd.melt(
                    tmp_df,
                    id_vars=["system", "ligand", "replica", "sample"],  # Columns to keep fixed
                    var_name="simulation",                   # Name for the new "variable" column
                    value_name="value"                  # Name for the new "value" column
                )
                
                
            else:
                raise RuntimeError
            BindFlowData = pd.concat([BindFlowData, tmp_df])

    BindFlowData = BindFlowData.pivot(index=["system", "ligand", "replica", "sample"], columns="simulation", values="value").reset_index()
    # Remove the columns' name
    BindFlowData.columns.name = None
    
    BindFlowData.sort_values(by=['system', 'ligand', "replica", "sample"], inplace=True)

    return BindFlowData.rename(columns=columns)

def get_exp_data():
    data = []
    for system in SYSTEM_ORDER:
        ref_data = get_plb_data(system, get_edges=False)
        ref_data["system"] = system
        data.append(ref_data)

    # Add a column to each DataFrame and concatenate
    ExpData = pd.concat(data, ignore_index=True)
    ExpData["replica"] = 1
    ExpData["sample"] = 1
    return ExpData

def get_external_data():
    # Khalak2021
    external_ref = "Khalak2021"
    df_Khalak2021 = pd.DataFrame()
    for system in ["p38", "tyk2"]:
        df = pd.read_csv(f"../../external-validation/{external_ref}/{system}.csv")
        # Convert all kJ/mol values to kcal/mol
        conversion_factor = 0.239005736
        for column in df.columns:
            if "kJ_mol" in column:
                df[column] = df[column] * conversion_factor
        df.rename(
            columns={
                "simulation_dG_value_kJ_mol": f"simulation_{external_ref}",
                "simulation_dG_error_kJ_mol": f"error_{external_ref}",
                "experimental_dG_value_kJ_mol": f"exp_{external_ref}",
                "experimental_dG_error_kJ_mol": f"exp_error_{external_ref}",
            },
            inplace=True
        )
        df["system"] = system
        df["replica"] = 1
        df["sample"] = 1
        df_Khalak2021 = pd.concat([df_Khalak2021, df])
    
    # Chen2023
    external_ref = "Chen2023"
    df_Chen2023 = pd.DataFrame()
    for system in ["mcl1", "p38", "ptp1b", "thrombin", "tyk2"]:
        if system == "p38":
            prefix_name = "lig_p38a_"
        else:
            prefix_name = "lig_"

        df = pd.read_excel(f"../../external-validation/{external_ref}/Congeneric_8proteins_ABFEP-internal-modification.xlsx", sheet_name=system)
        columns = {
            "Lig": "ligand",
            "dG_exp": f"exp_{external_ref}",
            "dG_RBFEP_21-3": f"simulation_{external_ref}-RBFEP",
            "21-3_run1": 1,
            "21-3_run2": 2,
            "21-3_run3": 3, 
        }
        df.rename(columns=columns, inplace=True)
        df["ligand"] = df["ligand"].apply(lambda row: f"{prefix_name}{row}")
        df["system"] = system
        df["sample"] = 1
        # Transform the wide-format DataFrame back to long format
        df = pd.melt(
            df,
            id_vars=["system", "ligand", "sample", f"exp_{external_ref}", f"simulation_{external_ref}-RBFEP"],  # Columns to keep fixed
            var_name="replica",                   # Name for the new "variable" column
            value_name=f"simulation_{external_ref}"                  # Name for the new "value" column
        )
        df_Chen2023 = pd.concat([df_Chen2023, df])


    # Lin2021
    external_ref = "Lin2021"
    df_Lin2021 = pd.DataFrame()
    prefix_name = "lig_"
    for system in ["p38", "tyk2"]:
        df = pd.read_excel(f"../../external-validation/{external_ref}/ci0c01329_si_002-internal-modification.xlsx", sheet_name=system)
        columns = {
            "Lig": "ligand",
            "dG_exp": f"exp_{external_ref}",
            "dG_value": f"simulation_{external_ref}",
            "dG_error": f"error_{external_ref}",
        }
        df.rename(columns=columns, inplace=True)
        df["ligand"] = df["ligand"].apply(lambda row: f"{prefix_name}{row.replace('-', '_')}")
        df["system"] = system
        df["replica"] = 1
        df["sample"] = 1
        df_Lin2021 = pd.concat([df_Lin2021, df])

    # Alibay2022
    external_ref = "Alibay2022"
    df_Alibay2022 = pd.read_csv(
        "https://raw.githubusercontent.com/IAlibay/fragment-opt-abfe-benchmark/refs/heads/main/free_energy_data/abfe.csv",
        usecols=["System", "ligand_ID", "run", "exp_dG", "calc_dG"]
        )
    columns = {
        "System": "system",
        "ligand_ID": "ligand",
        "run": "replica",
        "exp_dG": f"exp_{external_ref}",
        "calc_dG": f"simulation_{external_ref}",
    }
    df_Alibay2022.rename(columns=columns, inplace=True)
    df_Alibay2022 = df_Alibay2022[df_Alibay2022["system"].isin(["CycloD", "MCL-1"])]
    df_Alibay2022["system"] = df_Alibay2022["system"].map({"CycloD": "CyclophilinD", "MCL-1": "mcl1"})
    df_Alibay2022["ligand"] = df_Alibay2022.apply(
        lambda row: f"ligand-{row['ligand']}" if row["system"] == "CyclophilinD" else f"lig_{row['ligand']}",
        axis=1
    )
    df_Alibay2022["sample"] = 1


    # Li2019
    external_ref = "Li2019"
    columns = {
        "Ligand_ID": "ligand",
        "exp(kcal/mol)": f"exp_{external_ref}",
        "FEP-cal(kcal/mol)": f"simulation_mbar_{external_ref}",
        "MMPBSA-cal(kcal/mol)": f"simulation_mmpbsa_{external_ref}",
        "MMGBSA-cal(kcal/mol)": f"simulation_mmgbsa_{external_ref}",
    }
    df_Li2019 = pd.DataFrame()
    for system in ["thrombin", "tyk2"]:
        df = pd.read_excel(f"../../external-validation/{external_ref}/jm8b01763_si_002-manual.xlsx", sheet_name=system, usecols=columns.keys())
        df["system"] = system
        df.rename(columns=columns, inplace=True)
        df["ligand"] = df["ligand"].apply(lambda row: f"lig_{row}")
        df_Li2019 = pd.concat([df_Li2019, df])
    df_Li2019["replica"] = 1
    df_Li2019["sample"] = 1
    
    # Ries2024
    external_ref = "Ries2024"
    df_Ries2024 = pd.read_csv(
        "https://raw.githubusercontent.com/bigginlab/ABFE_workflow/refs/heads/main/examples/data/CyclophilinD_min/CyclophilinD_min_abfe_results.tsv",
        usecols=["ligand", "ABFE_mean", "ABFE_err"],
        delimiter="\t",
        )
    columns = {
        "ABFE_mean": f"simulation_{external_ref}",
        "ABFE_err": f"error_{external_ref}",
    }
    df_Ries2024.rename(columns=columns, inplace=True)
    df_Ries2024["system"] = "CyclophilinD"
    df_Ries2024["ligand"] = df_Ries2024.apply(
        lambda row: f"ligand-{row['ligand']}",
        axis=1
    )
    df_Ries2024["sample"] = 1
    df_Ries2024["replica"] = 1
    
    # Clark2024
    external_ref = "Clark2024"
    system = "CyclophilinD"
    df_Clark2024 = pd.read_csv(f"../../external-validation/{external_ref}/{system}.csv", index_col=0)
    df_Clark2024["system"] = system
    df_Clark2024["sample"] = 1
    df_Clark2024["replica"] = 1


    # Deflorian2020
    external_ref = "Deflorian2020"
    df = pd.read_csv("../../external-validation/Deflorian2020/A2A.csv", index_col=0)
    # Convert the 'edges' column from string to tuple
    df['edges'] = df['edges'].apply(ast.literal_eval)
    data = dict(zip(df.edges, df.dG))
    # Extract unique molecules
    molecules = set()
    for pair in data.keys():
        molecules.update(pair)
    molecules = sorted(molecules)

    # Map molecules to indices
    mol_index = {mol: i for i, mol in enumerate(molecules)}

    # Construct the matrix A and vector b
    n = len(molecules)
    m = len(data)
    A = np.zeros((m + 1, n))
    b = np.zeros(m + 1)

    # Populate the equations from data
    for row, ((mol1, mol2), delta_g) in enumerate(data.items()):
        A[row, mol_index[mol1]] = -1  # Coefficient for mol1
        A[row, mol_index[mol2]] = 1  # Coefficient for mol2
        b[row] = delta_g

    # Add the reference point for 4k
    A[-1, mol_index["4k"]] = 1
    b[-1] = ExpData[(ExpData["system"] == "A2A") &
                           (ExpData["ligand"] == "4k") &
                           (ExpData["replica"] == 1) &
                           (ExpData["sample"] == 1)]["exp_dG"].values[0]

    # Solve the system using least squares
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

    # Map back to molecule names
    absolute_g = {mol: x[mol_index[mol]] for mol in molecules}

    df_Deflorian2020 = pd.DataFrame(
        {
            "ligand": absolute_g.keys(),
            "simulation_Deflorian2020": absolute_g.values()
        }
    )
    df_Deflorian2020["system"] = "A2A"
    df_Deflorian2020["sample"] = 1
    df_Deflorian2020["replica"] = 1
    df_Deflorian2020["error_Deflorian2020"] = 0
    

    ExtData = pd.merge(df_Chen2023, df_Khalak2021, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Lin2021, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Alibay2022, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Li2019, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Ries2024, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Clark2024, on=["system", "ligand", "replica", "sample"], how="outer")
    ExtData = pd.merge(ExtData, df_Deflorian2020, on=["system", "ligand", "replica", "sample"], how="outer")

    return ExtData


BindFlowData_mbar = get_mbar_data()
BindFlowData_mmxbsa = get_mmxbsa_data()
ExpData = get_exp_data()
ExtData = get_external_data()

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality
Failed to patch pandas - unable to change molecule rendering
/Users/klimt/micromamba/envs/plbenchmark/lib/python3.11/site-packages/plbenchmark/ligands.py:189: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self._data["ROMol"].apply(lambda x: x[0])
Failed to patch pandas - unable to change molecule rendering
/Users/klimt/micromamba/envs/plbenchmark/lib/python3.11/site-packages/plbenchmark/ligands.py:189: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self._data["ROMol"].apply(lambda x: x[0])


In [2]:

BindFlowData = pd.merge(BindFlowData_mbar, BindFlowData_mmxbsa, on=["system", "ligand", "replica", "sample"], how="outer")
BindFlowData = pd.merge(BindFlowData, ExpData, on=["system", "ligand", "replica", "sample"], how="outer")
BindFlowData = pd.merge(BindFlowData, ExtData, on=["system", "ligand", "replica", "sample"], how="outer")

BindFlowData.to_csv("BindFlow.csv")

In [3]:
BindFlowData.columns

Index(['system', 'ligand', 'replica', 'simulation_mbar_espaloma-0.3.1',
       'simulation_mbar_gaff-2.11', 'simulation_mbar_openff-2.0.0', 'sample',
       'simulation_dg-c2-gb_espaloma-0.3.1', 'simulation_dg-c2-gb_gaff-2.11',
       'simulation_dg-c2-gb_openff-2.0.0',
       'simulation_dg-c2-pb_espaloma-0.3.1', 'simulation_dg-c2-pb_gaff-2.11',
       'simulation_dg-c2-pb_openff-2.0.0',
       'simulation_dg-ie-gb_espaloma-0.3.1', 'simulation_dg-ie-gb_gaff-2.11',
       'simulation_dg-ie-gb_openff-2.0.0',
       'simulation_dg-ie-pb_espaloma-0.3.1', 'simulation_dg-ie-pb_gaff-2.11',
       'simulation_dg-ie-pb_openff-2.0.0', 'simulation_dh-gb_espaloma-0.3.1',
       'simulation_dh-gb_gaff-2.11', 'simulation_dh-gb_openff-2.0.0',
       'simulation_dh-pb_espaloma-0.3.1', 'simulation_dh-pb_gaff-2.11',
       'simulation_dh-pb_openff-2.0.0', 'exp_dG', 'exp_dG_error',
       'exp_Chen2023', 'simulation_Chen2023-RBFEP', 'simulation_Chen2023',
       'simulation_Khalak2021', 'error_Khalak202

In [4]:
mean = BindFlowData.groupby(["system", "ligand"]).mean().reset_index().drop(columns=["replica", "sample"])
sem = BindFlowData.groupby(["system", "ligand"]).sem().reset_index().drop(columns=["replica", "sample"])

In [5]:
filter_df = mean[["system", "ligand" ,"exp_dG", "exp_dG_error", "simulation_dh-gb_espaloma-0.3.1", "simulation_dh-gb_gaff-2.11", "simulation_dh-gb_openff-2.0.0"]]
# One ligand was not done in espaloma, use espaloma instead of exp_dG
filter_df = filter_df[filter_df["simulation_dh-gb_espaloma-0.3.1"].notna()]
filter_df

,system,ligand,exp_dG,exp_dG_error,simulation_dh-gb_espaloma-0.3.1,simulation_dh-gb_gaff-2.11,simulation_dh-gb_openff-2.0.0
0,A2A,4g,-11.13,0.00,-39.479456,-36.206159,-40.494276
1,A2A,4h,-10.72,0.00,-38.727469,-38.002801,-41.481883
2,A2A,4i,-10.38,0.00,-38.654397,-37.191851,-39.355960
3,A2A,4j,-10.95,0.00,-38.986211,-38.227562,-41.604677
4,A2A,4k,-11.61,0.00,-41.055853,-38.822809,-41.212524
...,...,...,...,...,...,...,...
179,tyk2,lig_ejm_54,-10.63,0.18,-41.274888,-37.065309,-38.979720
180,tyk2,lig_ejm_55,-9.29,0.18,-37.452785,-34.436859,-37.666606
181,tyk2,lig_jmc_23,-11.81,0.18,-37.977831,-38.086578,-40.498543
182,tyk2,lig_jmc_27,-11.38,0.18,-41.309024,-38.189729,-41.359985


In [6]:
filter_df = mean[["system", "ligand" ,"exp_dG", "exp_Chen2023", "simulation_Chen2023", "simulation_mbar_gaff-2.11", "simulation_mbar_espaloma-0.3.1", "simulation_mbar_openff-2.0.0"]]
# One ligand was not done in espaloma, use espaloma instead of exp_dG
filter_df = filter_df[filter_df["exp_Chen2023"].notna() & (filter_df["exp_dG"].notna()) & (filter_df["system"] != "thrombin")]
filter_df[
    [
        "exp_dG", "exp_Chen2023",
        "simulation_mbar_espaloma-0.3.1",
        "simulation_mbar_gaff-2.11",
        "simulation_mbar_openff-2.0.0",
        "simulation_Chen2023",
    ]
].corr(method="pearson")

,exp_dG,exp_Chen2023,simulation_mbar_espaloma-0.3.1,simulation_mbar_gaff-2.11,simulation_mbar_openff-2.0.0,simulation_Chen2023
exp_dG,1.000000,0.999824,0.195790,0.285655,0.238305,0.510660
exp_Chen2023,0.999824,1.000000,0.184203,0.276861,0.227470,0.502455
simulation_mbar_espaloma-0.3.1,0.195790,0.184203,1.000000,0.880238,0.906222,0.821767
simulation_mbar_gaff-2.11,0.285655,0.276861,0.880238,1.000000,0.909084,0.857995
simulation_mbar_openff-2.0.0,0.238305,0.227470,0.906222,0.909084,1.000000,0.869807
simulation_Chen2023,0.510660,0.502455,0.821767,0.857995,0.869807,1.000000


In [7]:
filter_df[filter_df["system"] == "p38"]

,system,ligand,exp_dG,exp_Chen2023,simulation_Chen2023,simulation_mbar_gaff-2.11,simulation_mbar_espaloma-0.3.1,simulation_mbar_openff-2.0.0
87,p38,lig_p38a_2aa,-9.34,-9.27,-15.067400,-11.094771,-11.753859,-12.412487
88,p38,lig_p38a_2bb,-9.13,-9.06,-14.151100,-10.938932,-15.582165,-13.190718
90,p38,lig_p38a_2e,-10.78,-10.70,-16.254433,-11.222684,-14.284328,-12.917581
91,p38,lig_p38a_2ee,-12.35,-12.26,-16.302333,-13.012480,-15.006715,-14.083784
92,p38,lig_p38a_2f,-8.55,-8.48,-14.585967,-10.058724,-14.220466,-9.313782
93,p38,lig_p38a_2ff,-11.53,-11.44,-17.085833,-13.921024,-15.244097,-12.860461
94,p38,lig_p38a_2g,-10.74,-10.66,-16.471067,-14.411100,-14.670300,-14.095757
96,p38,lig_p38a_2gg,-10.74,-10.66,-15.779333,-12.233724,-12.208405,-10.810881
97,p38,lig_p38a_2h,-9.55,-9.48,-13.759433,-11.787466,-16.215890,-12.245260
98,p38,lig_p38a_2i,-10.13,-10.05,-14.631733,-12.982872,-11.597731,-11.233303
